# Mutable function parameters

- toc: true
- badges: true
- comments: true
- categories: [python]

From chapter 8 in [Fluent Python](https://www.oreilly.com/library/view/fluent-python/9781491946237/).

Functions that take mutable objects as arguments require caution, because function arguments are aliases for the passed arguments. This can cause two types of unintended behaviour:

- when setting a mutable object as defaults
- when aliasing a mutable object passed to the constructor.

## Setting a mutable object as default

In [13]:
class HauntedBus():
    
    def __init__(self, passengers=[]):
        self.passengers = passengers
        
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)
        

bus1 = HauntedBus(['pete', 'lara', 'nick'])
bus1.drop('nick')
print(bus1.passengers)

bus2 = HauntedBus()
bus2.pick('heather')
print(bus2.passengers)

bus3 = HauntedBus()
bus3.pick('katy')
print(bus3.passengers)

['pete', 'lara']
['heather']
['heather', 'katy']


This happens because the default list `[]` is evaluated when the function is defined, which means that we create a single empty list that is then aliased by all instances of bus to which we don't pass passenger list, buses 2 and 3 above. Because it is the same list, changes made to it either from bus2 or bus3 change the list of the other bus, too, because -- again -- it is the same list. (It is an example of why it matters of whether we think of variables/aliases as boxes or labels; the former cannot explain such behaviour, the latter can.)

In [18]:
HauntedBus.__init__.__defaults__

(['heather', 'katy'],)

The above shows that after the `bus3.pick('katy')` call above, the default list is now changed, and will be inherited by future instances of `HauntedBus`.

In [19]:
bus4 = HauntedBus()
bus4.passengers

['heather', 'katy']

What to do instead? The solution is to create a new empty list each time no list is provided.

In [23]:
class Bus():
    
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
            
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)
        
bus1 = Bus()
bus1.pick('tim')
bus2 = Bus()
bus2.passengers

[]

## Aliasing a mutable object argument inside the function

The `init` method of the above class copies the passed passenger list by calling `list(passengers)`. This is critical. If, instead of copying we alias the passed list, we change lists defined outside the function that are passed as arguments, which is probably not what we want.

In [24]:
class Bus():
    
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = passengers
            
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)
        
team = ['hugh', 'lisa', 'gerd', 'adam', 'emily']

bus = Bus(team)
bus.drop('hugh')
team

['lisa', 'gerd', 'adam', 'emily']

Again, the reason for this is that `self.passengers` is an alias for `passengers`, which is itself an alias for `team`, so that all operations we perfom on `self.passengers` are actually performed on `team`. The identity check below shows what the passengers attribute of `bus` is indeed the same object as the team list.

In [28]:
bus.passengers is team

True

**To summarise:** unless there is a good reason for an exception, for functions that take mutable objects as arguments do the following:

1. Create a new empty list each time a class is instantiated rather than at function definition (usually when the class is loaded) by using None as the default parameter.

2. Make a copy of the mutable object for processing inside the function to leave the original object unchanged.

## Main sources

- [Fluent Python](https://www.oreilly.com/library/view/fluent-python/9781491946237/)
- [Python Cookbook](https://www.oreilly.com/library/view/python-cookbook-3rd/9781449357337/)
- [Learning Python](https://www.oreilly.com/library/view/learning-python-5th/9781449355722/)